# Supply Chain Analytics

Transform supply chain activities from guessing, to ones that makes decision using data. An essential tool in Supply Chain Analytics is using optimization analysis to assist in decision making. According to Deloitte, 79% of organizations with high performing supply chains achieve revenue growth that is significantly above average. 

This notebook will introduce you to `PuLP`, a Linear Program optimization modeler written in Python. Using `PuLP`, the notebook will show you how to formulate and answer Supply Chain optimization questions such as where a production facility should be located, how to allocate production demand across different facilities, and more. 

We will explore the results of the models and their implications through sensitivity and simulation testing. This notebook will help you position yourself to improve the decision making of a supply chain by leveraging the power of Python and `PuLP`.


Linear Programming (LP) is a key technique for Supply Chain Optimization. The `PuLP` framework is an easy to use tool for working with LP problems and allows the programmer to focus on modeling. 

We learn the basics of LP problems and start to learn how to use the `PuLP` framework to solve them.

<img src= '../images/lp.jpg'>

## To LP, or to not IP?
In this exercise you are optimizing the manufacturing costs over a manufacturing site. The site only manufactures two types of hammers, types A and B. Considering the objective function and constraints listed below, what type of modeling optimization is this?

- Minimize: `0.32*A + 0.74*B`

- Subject to:
    - `A ≥ 0`
    - `B ≥ 0`
    - `A + 6B ≥ 70`
    
- A = Number of individual hammers of type A
- B = Number of individual hammers of type B

Since our decision variables are the number of hammers they must be discrete variables. You are not manufacturing partial hammers.

# Simple resource scheduling exercise
In this exercise you are planning the production at a glass manufacturer. This manufacturer only produces wine and beer glasses:

- there is a maximum production capacity of 60 hours
- each batch of wine and beer glasses takes 6 and 5 hours respectively
- the warehouse has a maximum capacity of 150 rack spaces
- each batch of the wine and beer glasses takes 10 and 20 spaces respectively
- the production equipment can only make full batches, no partial batches

Also, we only have orders for 6 batches of wine glasses. Therefore, we do not want to produce more than this. Each batch of the wine glasses earns a profit of usd5 and the beer usd4.5.

The objective is to maximize the profit for the manufacturer.

In [8]:
from pulp import LpProblem, LpMaximize, LpVariable

Initializes the model using the `LpProblem()` function. Remember you want to model the objective function as the amount of profit earned.

In [6]:
# Initialize Class
model = LpProblem("Maximize_Glass_Profit", LpMaximize)

Define the model variables for the number of wine and beer produced. In terms of profit, the sky is the limit, but we definitely want to ensure we're not losing money.

In [9]:
# Define Decision Variables
wine = LpVariable('Wine', lowBound=0, upBound=None, cat='Integer')
beer = LpVariable('Beer', lowBound=0, upBound=None, cat='Integer')

Write the objective function. How does each wine and beer glass produced affect profit?

In [10]:
# Define Objective Function
model += 5 * wine + 4.5 * beer

Define the 3 model constraints. Think about the production capacity, reflect on the warehouse capacity, and don't forget about the current amount of orders. The 3 constraints:

- One limits the total amount of production based on hours to less than or equal to 60.
- One limits the total amount of production based the total rack spaces in the warehouse of less than or equal to 150
- One limits wine glass production based on current demand of less than equal to 6 batches.

In [11]:
# Define Constraints
model += 6 * wine + 5 * beer <= 60
model += 10 * wine + 20 * beer <= 150
model += wine <= 6

In [12]:
# Solve Model
model.solve()

1

In [13]:
print("Produce {} batches of wine glasses".format(wine.varValue))
print("Produce {} batches of beer glasses".format(beer.varValue))

Produce 6.0 batches of wine glasses
Produce 4.0 batches of beer glasses


Awesome job! Using `LpMaximize()` makes sense because we are trying to maximize profits. It does not make sense to set an upper bound on the number of wine or beer glasses produced: the more the better. However, setting a lower bound is important because producing negative glasses is impossible. Also, selecting the variable to be an integer makes sense because we can not make half batches. Adding the last constraint on wine based on current orders is important to add to avoid producing too many wine glasses.

# Logistics planning problem
You are consulting for kitchen oven manufacturer helping to plan their logistics for next month. There are two warehouse locations (New York, and Atlanta), and four regional customer locations (East, South, Midwest, West). The expected demand next month for East it is 1,800, for South it is 1,200, for the Midwest it is 1,100, and for West it is 1000. The cost for shipping each of the warehouse locations to the regional customer's is listed in the table below. Your goal is to fulfill the regional demand at the lowest price.

|Customer |	New York |	Atlanta
| -- | -- | -- |
| East |	211 |	232
| South |	232 |	212
| Midwest |	240 |	230
| West |	300 |	280

In [24]:
from pulp import *

Two Python dictionaries `costs` and `var_dict` have been created for you containing the costs and decision variables of the model. You can explore them in the console.

In [18]:
costs = {('Atlanta', 'East'): 232,
         ('Atlanta', 'Midwest'): 230,
         ('Atlanta', 'South'): 212,
         ('Atlanta', 'West'): 280,
         ('New York', 'East'): 211,
         ('New York', 'Midwest'): 240,
         ('New York', 'South'): 232,
         ('New York', 'West'): 300}

In [21]:
# Define Decision Variables
atle = LpVariable('atle', lowBound=0, upBound=None, cat='Integer')
atlm = LpVariable('atlm', lowBound=0, upBound=None, cat='Integer')
atls = LpVariable('atls', lowBound=0, upBound=None, cat='Integer')
atlw = LpVariable('atlw', lowBound=0, upBound=None, cat='Integer')
ne = LpVariable('ne', lowBound=0, upBound=None, cat='Integer')
nm = LpVariable('nm', lowBound=0, upBound=None, cat='Integer')
ns = LpVariable('ns', lowBound=0, upBound=None, cat='Integer')
nw = LpVariable('nw', lowBound=0, upBound=None, cat='Integer')

In [22]:
var_dict = {('Atlanta', 'East'): atle,
         ('Atlanta', 'Midwest'): atlm,
         ('Atlanta', 'South'): atls,
         ('Atlanta', 'West'): atlw,
         ('New York', 'East'): ne,
         ('New York', 'Midwest'): nm,
         ('New York', 'South'): ns,
         ('New York', 'West'): nw}

In [25]:
# Initialize Model
model = LpProblem("Minimize_Transportation_Costs", LpMinimize)

Create a list for the `warehouse` locations, the regional `customers` and the `regional_demand` respectively, making sure the demand and customer indices match, and zip the `customers` and `regional_demand` lists to build the `demand` dictionary.

In [26]:
# Build the lists and the demand dictionary
warehouse = ['New York', 'Atlanta']
customers = ['East', 'South', 'Midwest', 'West']
regional_demand = [1800, 1200, 1100, 1000]

demand = dict(zip(customers, regional_demand))
demand

{'East': 1800, 'South': 1200, 'Midwest': 1100, 'West': 1000}

Our decision variables in this exercise are the number of shipments for each warehouse and customer combination. You want to minimize the total cost of shipping. The total cost of shipping is the shipping costs along a route multiplied by the number of shipments along that route. Define the objective function using `lpSum()` with list comprehension, using the `var_dict` dictionary that was created for you, iterating over customers.

In [27]:
# Define Objective
model += lpSum([costs[(w, c)] * var_dict[(w, c)] 
                for c in customers for w in warehouse])

Define the constraint using `lpSum`. Loop through the list of `customers`, and for each sum the shipments from the warehouses and set it equal to the customer demand. Remember we now have the customer demand as a dictionary with the customer name as the key.

In [28]:
# For each customer, sum warehouse shipments and set equal to customer demand
for c in customers:
    model += lpSum([var_dict[(w, c)] for w in warehouse]) == demand[c]

In [29]:
# Solve Model
model.solve()

1

In [33]:
print("atle = {}".format(atle.varValue))
print("atlm = {}".format(atlm.varValue))
print("atls = {}".format(atls.varValue))
print("atlw = {}".format(atlw.varValue))
print("ne = {}".format(ne.varValue))
print("nm = {}".format(nm.varValue))
print("ns = {}".format(ns.varValue))
print("nw = {}".format(nw.varValue))

atle = 0.0
atlm = 1100.0
atls = 1200.0
atlw = 1000.0
ne = 1800.0
nm = 0.0
ns = 0.0
nw = 0.0


# Scheduling workers problem
You are looking to hire workers to work in a warehouse. Each worker is expected to work 5 consecutive days and then have two days off. The chart below has the estimated number of workers you will need each day. You are looking to hire the minimum number of workers to handle the workload for each day.

Expected Workload:

| Day of Week	| Employees Needed |
| -- | -- |
| 0 = Monday	| 31 |
| 1 = Tuesday	| 45 | 
| 2 = Wednesday	| 40 |
| 3 = Thursday	| 40 |
| 4 = Friday	| 48 |
| 5 = Saturday	| 30 |
| 6 = Sunday	| 25 |

In [34]:
model = LpProblem("Minimize Staffing", LpMinimize)

In [35]:
days = list(range(7))
x = LpVariable.dicts('staff_', days, lowBound=0, cat='Integer')

In [36]:
model += lpSum([x[i] for i in days])

Define the seven constraints on the number of workers working on a day. There is a pattern to the constraints. Think about what days a worker would be off if they started on Mon., Tue., Wed., ….

In [37]:
# Define Constraints
model += x[0] + x[3] + x[4] + x[5] + x[6] >= 31
model += x[0] + x[1] + x[4] + x[5] + x[6] >= 45
model += x[0] + x[1] + x[2] + x[5] + x[6] >= 40
model += x[0] + x[1] + x[2] + x[3] + x[6] >= 40
model += x[0] + x[1] + x[2] + x[3] + x[4] >= 48
model += x[1] + x[2] + x[3] + x[4] + x[5] >= 30
model += x[2] + x[3] + x[4] + x[5] + x[6] >= 25

In [38]:
model.solve()

1

In [48]:
total = 0
for i, v in x.items():
    print(f'Day of the week {i} = {v.varValue}')
    total += v.varValue
print(f'Total workers {total}')

Day of the week 0 = 14.0
Day of the week 1 = 14.0
Day of the week 2 = 8.0
Day of the week 3 = 0.0
Day of the week 4 = 13.0
Day of the week 5 = 0.0
Day of the week 6 = 4.0
Total workers 53.0


# Logical constraint exercise
Your customer has ordered six products to be delivered over the next month. You will need to ship multiple truck loads to deliver all of the products. There is a weight limit on your trucks of 25,000 lbs. For cash flow reasons you desire to ship the most profitable combination of products that can fit on your truck.

| Product	| Weight (lbs) |	Profitability ($US) |
| --- | --- | --- |
| A	| 12,583 |	102,564 |
| B	| 9,204 |	130,043 |
| C	| 12,611 |	127,648 |
| D	| 12,131 |	155,058 |
| E	| 12,889 |	238,846 |
| F	| 11,529 |	197,030 |

In [50]:
weight = {'A':12583,
         'B':9204,
         'C':12611,
         'D':12131,
         'E':12889,
         'F':11529}

In [51]:
prof = {'A':102564,
         'B':130043,
         'C':127648,
         'D':155058,
         'E':238846,
         'F':197030}

In [52]:
prod = ['A','B','C','D','E','F']

In [53]:
# Initialized model
model = LpProblem("Loading_Truck_Problem", LpMaximize)

C:\Users\vilieri.i\AppData\Local\Continuum\anaconda3\lib\site-packages\pulp\pulp.py:1195: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [54]:
# Defined decision variables
x = LpVariable.dicts('ship_', prod, cat='Binary')

In [55]:
# Define Objective
model += lpSum([prof[i] * x[i] for i in prod])

In [59]:
# Define Constraint
model += lpSum([weight[i] * x[i] for i in prod]) <= 25000
model += x['D'] + x['E'] + x['F'] <= 1

In [60]:
model.solve()

1

In [61]:
for i in prod:
    print("{} status {}".format(i, x[i].varValue))

A status 0.0
B status 1.0
C status 0.0
D status 0.0
E status 1.0
F status 0.0
